In [30]:
import os
import json
import nltk
from gensim import models
import numpy as np
from gensim import corpora, models, similarities
import pickle
import numpy as np

model = models.Word2Vec.load('word2vec.bin')

In [33]:
with open('conversation.json', 'r') as f:
    data = json.load(f)
    cor = data["conversations"]

In [34]:
#Seperating Questions and answers into x and y respectively
x = []
y = []

for i in range(len(cor)):
    for j in range(len(cor[i])-1):
        x.append(cor[i][j])
        y.append(cor[i][j+1])

In [35]:
x[:10]

['Good morning, how are you?',
 'I am doing well, how about you?',
 "I'm also good.",
 "That's good to hear.",
 'Hello',
 'Hi',
 'How are you doing?',
 'I am doing well.',
 'That is good to hear',
 'Yes it is.']

In [36]:
y[:10]

['I am doing well, how about you?',
 "I'm also good.",
 "That's good to hear.",
 'Yes it is.',
 'Hi',
 'How are you doing?',
 'I am doing well.',
 'That is good to hear',
 'Yes it is.',
 'Can I help you with anything?']

In [37]:
#Tokenizing all the sentences in the dataset 
tok_x = []
tok_y = []

for i in range(len(x)):
    tok_x.append(nltk.word_tokenize(x[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))

In [38]:
tok_x[:10]

[['good', 'morning', ',', 'how', 'are', 'you', '?'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', "'m", 'also', 'good', '.'],
 ['that', "'s", 'good', 'to', 'hear', '.'],
 ['hello'],
 ['hi'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', '.'],
 ['that', 'is', 'good', 'to', 'hear'],
 ['yes', 'it', 'is', '.']]

In [47]:
#Removing punctuation characters and converting each word into vector using word2vec model

vec_x = []
vec_y = []
from string import punctuation

for sent in tok_x:
    sentvec = [w for w in sent if w not in punctuation]
    sentvec1 = [model[w] for w in sentvec if w in model.wv.vocab]    
    vec_x.append(sentvec1)

for sent in tok_y:
    sentvec = [w for w in sent if w not in punctuation]
    sentvec1 = [model[w] for w in sentvec if w in model.wv.vocab]    
    vec_y.append(sentvec1)

/home/spiringseeker57/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/home/spiringseeker57/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [48]:
#Clipping each sentence to length 15 and padding with a vector of
#ones to th sentences with length<15

sentend = np.ones((300,), dtype=np.float32)

for tok_sent in vec_x:
    tok_sent[14: ] =[]
    tok_sent.append(sentend)
    
for tok_sent in vec_x:
    if(len(tok_sent)<15):
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)
            
for tok_sent in vec_y:
    tok_sent[14: ] =[]
    tok_sent.append(sentend)
    
for tok_sent in vec_y:
    if(len(tok_sent)<15):
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)

In [49]:
#Saving the preprocessed data into a pickle file
with open('conversation.pickle','wb') as f:
    pickle.dump([vec_x,vec_y],f)